In [1]:
import os
os.chdir('../')
%pwd

'/home/sliti-wassim/MLOPS Bootcamp/MLops_Code/Projects/DataScienceProject-1'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [10]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml , create_directories

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config       

In [17]:
import  urllib.request as  request
from src.DataScience import logger
import zipfile
import os

In [18]:
## Components-Data Ingestion
class DataIngestion :
    def __init__(self,
                 config:DataIngestionConfig):
        self.config = config
    
    ## Downloading the zip file 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded ! with following info \n{header}") 
        else :
            logger.info(f"{filename} File already exists !! ")
    
    ## Unzip file
    def extract_zipfile(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref :
            zip_ref.extractall(unzip_path)




### Execute Data Ingestion

In [19]:
try :
    config = ConfigurationManager()
    data_ingestion_configuration = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_configuration)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

except Exception as e:
    raise e    

[2025-03-30 03:55:58,082: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-30 03:55:58,084: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-30 03:55:58,085: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-30 03:55:58,085: INFO: common: created directory at: artifact]
[2025-03-30 03:55:58,086: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-30 03:55:59,826: INFO: 4057660030: artifacts/data_ingestion/data.zip downloaded ! with following info 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6A5A:25C996:30EFB79:3789861:67E8B2BE
Accept-Ranges: bytes
Date: Sun,